In [1]:
#natural language toolkit 
import pandas as pd

In [2]:
df = pd.read_csv('spam.csv', encoding='latin-1')
df = df[['v1', 'v2']]

In [3]:
df.columns = ['Spam', 'Mensagem']

In [4]:
df.sample(5)

,Spam,Mensagem
2664,ham,He remains a bro amongst bros
1931,ham,What pa tell me.. I went to bath:-)
65,spam,"As a valued customer, I am pleased to advise y..."
2528,ham,jay says he'll put in &lt;#&gt;
743,ham,Ok. There may be a free gym about.


In [5]:
# explorando como funciona o countVectorizer
msg = 'What is your record for one night? :) bananinha'

#bag-of-words
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
vect.fit([msg])

print(vect.get_feature_names_out())

vect.transform([
    'What is your your your :) record for one night? :)'
]).toarray()

pd.DataFrame(
    data = vect.transform([
        'What is your your your :) record for one night? :)',
        'What record for one night? :)',
        'What record for one naoexisto record? :)',
    ]).toarray(),
    columns = vect.get_feature_names_out()
)

['bananinha' 'for' 'is' 'night' 'one' 'record' 'what' 'your']


,bananinha,for,is,night,one,record,what,your
0,0,1,1,1,1,1,1,3
1,0,1,0,1,1,1,1,0
2,0,1,0,0,1,2,1,0


In [6]:
def processa_spam(linha):
    """Recebe uma linha e retorna `0` caso seja um `ham` ou `1` caso seja `spam`
       `linha` é uma linha de um dataframe.
    """

    if linha['Spam']=='ham':
        return 0
    else: # Se for spam
        return 1

def tamanho_mensagem(linha):
    return len(linha['Mensagem'])

df['Spam (num)'] = df.apply(processa_spam, axis=1)
df['Tamanho']    = df.apply(tamanho_mensagem, axis=1)

df

,Spam,Mensagem,Spam (num),Tamanho
0,ham,"Go until jurong point, crazy.. Available only ...",0,111
1,ham,Ok lar... Joking wif u oni...,0,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155
3,ham,U dun say so early hor... U c already then say...,0,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,61
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1,161
5568,ham,Will Ì_ b going to esplanade fr home?,0,37
5569,ham,"Pity, * was in mood for that. So...any other s...",0,57
5570,ham,The guy did some bitching but I acted like i'd...,0,125


In [7]:
# stopwords (p.ex. the, and, a, an, or, so, like, )
# (em bag of words, tem palavras que não agregam muito)
# Lemmatization stemming também existem (mas não veremos aqui)

In [9]:
#!pip install nltk
from nltk.corpus import stopwords

def processa_mensagem(linha):
    msg = linha["Mensagem"]
    msg = msg.lower()

    # Vamos separar só as palavras que não são stopwords (e jogar fora o resto)
    palavras_filtradas = []
    for palavra in msg.split(' '):
        if palavra not in stopwords.words("english"): # a palavra não é uma stopword
            palavras_filtradas.append(palavra)
            
    msg_sem_stopwords = ' '
    for palavra in palavras_filtradas:
        msg_sem_stopwords = msg_sem_stopwords + ' ' + palavra
    
    # Versão "pythonica" do mesmo código acima
    msg_sem_stopwords = " ".join([palavra for palavra in msg.split(" ")
                                  if palavra not in stopwords.words("english")])
    
    return msg_sem_stopwords

df["Mensagem"] = df.apply(processa_mensagem, axis=1)
    

In [44]:
X, y = df['Mensagem'], df['Spam (num)']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

vect = CountVectorizer() # não usamos os dados de teste nele!
vect.fit(X_train)

print(vect.get_feature_names_out()[1500:1510])
X_train = vect.transform(X_train)
X_test  = vect.transform(X_test)


['buns' 'burden' 'burger' 'burgundy' 'burn' 'burning' 'burns' 'burnt'
 'bus' 'bus8']


In [46]:
# futuramente veremos o tf-idf
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [47]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, clf.predict(X_test))

0.9827709978463748

In [52]:
mensagem_teste = "CONGRATULATIONS! YOU ARE A WINNER you earned 1,000,000US DOLLARS for free you just need to answer this with your credit card information"

print(clf.predict(vect.transform([mensagem_teste])))

mensagem_teste2 = """
Dear Decision maker ; This letter was specially selected 
to be sent to you . If you no longer wish to receive 
our publications simply reply with a Subject: of "REMOVE" 
and you will immediately be removed from our mailing 
list ! This mail is being sent in compliance with Senate 
bill 1623 ; Title 2 ; Section 303 . This is not a get 
rich scheme ! Why work for somebody else when you can 
become rich in 20 WEEKS . Have you ever noticed society 
seems to be moving faster and faster and people love 
convenience ! Well, now is your chance to capitalize 
on this . WE will help YOU SELL MORE plus process your 
orders within seconds . You can begin at absolutely 
no cost to you ! But don't believe us ! Ms Simpson 
of Wyoming tried us and says "My only problem now is 
where to park all my cars" . We are licensed to operate 
in all states . You will blame yourself forever if 
you don't order now . Sign up a friend and your friend 
will be rich too ! Warmest regards !"""

print(clf.predict(vect.transform([mensagem_teste2])))

[1]
[0]
